In [59]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Case study: matrix multiplications

GEMM that computes C = A $\times$ B is the core of many AI/ML applications. The most naive implementation of GEMM takes $O(n^3)$. Assume it takes 1 second to perform GEMM on 1,024$\times$1,024$\times$1,024 matrices. How much time do you expect it would take for 2,048$\times$2,048$\times$2,048 matrices?

In [2]:
render_code("matrix_mul/mm.c", show=["//START","//END"])

// matrix_mul/mm.c:71-92 (22 lines)
//START
void mm(double **a, double **b, double **c)
{
  int i,j,k;
  for(i = 0; i < ARRAY_SIZE; i++)
  {
    for(j = 0; j < ARRAY_SIZE; j++)
    {
      for(k = 0; k < ARRAY_SIZE; k++)
      {
        c[i][j] += a[i][k]*b[k][j];
        #ifdef DUMP
          fprintf(stderr, "a[%d][%d], %p\n",i,k, &a[i][k]);
          fprintf(stderr, "b[%d][%d], %p\n",k,j, &b[k][j]);
          fprintf(stderr, "c[%d][%d], %p\n",i,j, &c[i][j]);
        #endif
      }
    }
  }
  return;
}
//END

In [19]:
! cd matrix_mul; make clean; make mm

rm -f blockmm mm blockmm_transpose cachegrind.* mm_dump
gcc -DHAVE_LINUX_PERF_EVENT_H -O3 mm.c perfstats.c -o mm 


In [85]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > mm.csv
! ./matrix_mul/mm 512 >> ./matrix_mul/mm.csv ;./matrix_mul/mm 1024 >> ./matrix_mul/mm.csv ; ./matrix_mul/mm 2048 >> ./matrix_mul/mm.csv
#! cs203 job memory "./matrix_mul/mm 1024 >> ./matrix_mul/mm.csv ; ./matrix_mul/mm 2048 >> ./matrix_mul/mm.csv"

In [21]:
display_df_mono(render_csv("matrix_mul/mm.csv"))

,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,1077421136,740950660,0.687708,0.213694,0.158337,0.242821,130571299,537727247
1,8621075972,11060150164,1.282920,0.192954,2.134102,0.235004,1011566024,4304457286
2,68999463366,117406516775,1.701557,0.192907,22.648542,0.312074,10752614433,34455299547


WOW! Compuational complexty breaks again! The GEMM performance go wild because of cache misses!

What kind of misses are we seeing?

In [83]:
! make -C matrix_mul mm_dump; ./matrix_mul/mm_dump 256 >& mm_dump_address.csv

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/matrix_mul'
gcc -DHAVE_LINUX_PERF_EVENT_H -DDUMP -O3 mm.c perfstats.c -o mm_dump 
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/matrix_mul'


In [84]:
! echo "element,address" > mm_dump_addresses_digest.csv 
! head -n 101 mm_dump_address.csv | grep "b\[" >> mm_dump_addresses_digest.csv
df = pd.read_csv("mm_dump_addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

,element,address,tag,index
0,b[0][0],0x7ccb5d0ff000,0x7ccb5d0ff,0x0
1,b[1][0],0x7ccb5d0ff800,0x7ccb5d0ff,0x20
2,b[2][0],0x7ccb5d100000,0x7ccb5d100,0x0
3,b[3][0],0x7ccb5d100800,0x7ccb5d100,0x20
4,b[4][0],0x7ccb5d101000,0x7ccb5d101,0x0
5,b[5][0],0x7ccb5d101800,0x7ccb5d101,0x20
6,b[6][0],0x7ccb5d102000,0x7ccb5d102,0x0
7,b[7][0],0x7ccb5d102800,0x7ccb5d102,0x20
8,b[8][0],0x7ccb5d103000,0x7ccb5d103,0x0
9,b[9][0],0x7ccb5d103800,0x7ccb5d103,0x20


### Matrix tiling algorithm

Let's try to partition GEMM into smaller tiles!

In [122]:
render_code("matrix_mul/blockmm.c", show=["//START","//END"])

// matrix_mul/blockmm.c:59-77 (19 lines)
//START
void blockmm(double **a, double **b, double **c, uint64_t M, uint64_t N, uint64_t K)
{
  uint64_t i,j,k, ii, jj, kk;
  for(i = 0; i < M; i+=tile_size)
  {
    for(j = 0; j < K; j+=tile_size)
    {
      for(k = 0; k < N; k+=tile_size)
      {        
          for(ii = i; ii < i+tile_size; ii++)
            for(jj = j; jj < j+tile_size; jj++)
              for(kk = k; kk < k+tile_size; kk++)
                c[ii][jj] += a[ii][kk]*b[kk][jj];
      }
    }
  }  
}
//END

In [124]:
! cd matrix_mul/; make blockmm

gcc -O3 -DHAVE_LINUX_PERF_EVENT_H blockmm.c perfstats.c -o blockmm 
blockmm.c: In function ‘main’:
blockmm.c:48:16: warning: format ‘%lu’ expects argument of type ‘long unsigned int’, but argument 3 has type ‘int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
   48 |   printf("%d,%lu,",ARRAY_SIZE,tile_size);
      |              ~~^              ~~~~~~~~~
      |                |              |
      |                |              int
      |                long unsigned int
      |              %u


In [125]:
! cd matrix_mul; echo "size,tile_size,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm.csv
! ./matrix_mul/blockmm 512 8 >> ./matrix_mul/blockmm.csv ;./matrix_mul/blockmm 1024 8 >> ./matrix_mul/blockmm.csv ; ./matrix_mul/blockmm 2048 8 >> ./matrix_mul/blockmm.csv; ./matrix_mul/blockmm 4096 8 >> ./matrix_mul/blockmm.csv

In [126]:
display_df_mono(render_csv("matrix_mul/mm.csv"))
display_df_mono(render_csv("matrix_mul/blockmm.csv"))

,index,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,512,1061097291,773717636,0.729167,0.226353,0.175133,0.238468,126278787,529541442
1,1024,8619156556,10039092055,1.164742,0.193245,1.940008,0.232634,1001195428,4303733009
2,2048,69003214171,123227533479,1.785823,0.193324,23.822892,0.317829,10950993986,34455669245


,index,size,tile_size,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,0,512,8,1184934085,249407711,0.210482,0.289598,0.072228,0.008992,5016069,557809112
1,1,1024,8,10129925601,2199588954,0.217138,0.193574,0.425784,0.009332,44499631,4768646456
2,2,2048,8,81046611334,21430097264,0.264417,0.193320,4.142870,0.010273,391948242,38151812189
3,3,4096,8,648451267176,215434017013,0.332229,0.193189,41.619418,0.012172,3715273312,305242595577


In [128]:
! ./matrix_mul/blockmm 2048 4 >> ./matrix_mul/blockmm.csv
! ./matrix_mul/blockmm 2048 16 >> ./matrix_mul/blockmm.csv 
! ./matrix_mul/blockmm 2048 32 >> ./matrix_mul/blockmm.csv 
display_df_mono(render_csv("matrix_mul/blockmm.csv"))

,index,size,tile_size,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,0,512,8,1184934085,249407711,0.210482,0.289598,0.072228,0.008992,5016069,557809112
1,1,1024,8,10129925601,2199588954,0.217138,0.193574,0.425784,0.009332,44499631,4768646456
2,2,2048,8,81046611334,21430097264,0.264417,0.193320,4.142870,0.010273,391948242,38151812189
3,3,4096,8,648451267176,215434017013,0.332229,0.193189,41.619418,0.012172,3715273312,305242595577
4,4,2048,4,97764764568,24002313658,0.245511,0.193272,4.638978,0.014966,653117873,43641172286
5,5,2048,16,74379556398,19139916956,0.257328,0.193848,3.710233,0.071820,2589801707,36059829859
6,6,2048,32,71542461032,27805205223,0.388653,0.193224,5.372632,0.217134,7646177423,35214019094
7,7,2048,4,97769807492,26308542322,0.269087,0.193414,5.088442,0.014713,642099154,43643056766
8,8,2048,16,74474073583,19865740180,0.266747,0.193115,3.836382,0.072886,2631590226,36105448834
9,9,2048,32,71543283237,27806966234,0.388673,0.193265,5.374123,0.216973,7640559416,35214360073


In [114]:
render_code("matrix_mul/blockmm_transpose.c", show=["//START","//END"])

// matrix_mul/blockmm_transpose.c:61-79 (19 lines)
//START
void blockmm_transpose(double **a, double **b, double **c, uint64_t M, uint64_t N, uint64_t K)
{
  int i,j,k, ii, jj, kk;
  for(i = 0; i < M; i+=tile_size)
  {
    for(j = 0; j < K; j+=tile_size)
    {
      for(k = 0; k < N; k+=tile_size)
      {        
          for(ii = i; ii < i+tile_size; ii++)
            for(jj = j; jj < j+tile_size; jj++)
              for(kk = k; kk < k+tile_size; kk++)
                c[ii][jj] += a[ii][kk]*b[jj][kk];
      }
    }
  }  
}
//END

### Matrix transpose

In [115]:
! cd matrix_mul; make blockmm_transpose; echo "size,tile_size,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm_transpose.csv
! ./matrix_mul/blockmm_transpose 512 8 >> ./matrix_mul/blockmm_transpose.csv ;./matrix_mul/blockmm_transpose 1024 8 >> ./matrix_mul/blockmm_transpose.csv ; ./matrix_mul/blockmm_transpose 2048 8 >> ./matrix_mul/blockmm_transpose.csv; ./matrix_mul/blockmm_transpose 4096 8 >> ./matrix_mul/blockmm_transpose.csv

make: 'blockmm_transpose' is up to date.


In [116]:
! ./matrix_mul/blockmm_transpose 2048 16 >> ./matrix_mul/blockmm_transpose.csv 
! ./matrix_mul/blockmm_transpose 2048 32 >> ./matrix_mul/blockmm_transpose.csv 
! ./matrix_mul/blockmm_transpose 2048 64 >> ./matrix_mul/blockmm_transpose.csv 

In [117]:
display_df_mono(render_csv("matrix_mul/blockmm_transpose.csv"))

,index,size,tile_size,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,0,512,8,1073418342,245138471,0.228372,0.245119,0.060088,0.006866,3013988,438954706
1,1,1024,8,8793730352,1984512186,0.225674,0.194710,0.386404,0.002296,8254883,3596110071
2,2,2048,8,70352538520,16012980786,0.227611,0.193229,3.094171,0.001973,56770219,28770337440
3,3,4096,8,562445621466,129127888984,0.229583,0.193484,24.984207,0.001984,456280784,230011425587
4,4,2048,16,64810073062,15221864848,0.234869,0.193117,2.939604,0.024597,665236375,27045326530
5,5,2048,32,62383258379,14915042199,0.239087,0.193462,2.885493,0.041058,1082281914,26359904976
6,6,2048,64,61244625454,13670733854,0.223215,0.193268,2.642110,0.023971,624600176,26056559695


In [118]:
display_df_mono(render_csv("matrix_mul/blockmm.csv"))

,index,size,tile_size,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,0,512,8,1254565128,272074259,0.216867,0.297643,0.080981,0.009281,5481123,590564358
1,1,1024,8,10129836406,2196090695,0.216794,0.193114,0.424096,0.009275,44227690,4768621106
2,2,2048,8,80996985555,21043742544,0.259809,0.193444,4.070776,0.010135,386443456,38128531445
3,3,4096,8,648451975425,214652641152,0.331023,0.193265,41.484849,0.012207,3726146960,305242525287
4,4,2048,4,97765686275,24149510064,0.247014,0.193189,4.665430,0.015102,659053606,43641501149
5,5,2048,16,74473114435,19369857501,0.260092,0.193204,3.742332,0.071790,2591990240,36105122733
6,6,2048,32,71543334296,27812871208,0.388756,0.193112,5.371009,0.217011,7641891998,35214370198


## Prefetch

x86 provide prefetch instructions. As a programmer, you may insert ```_mm_prefetch``` in x86 programs to perform software prefetch for your code. The gcc compiler also has a flag ```-fprefetch-loop-arrays``` to automatically insert software prefetch instructions.

### Using prefetch in matrix transpose code

The following example is a highly optimized matrix transpose code. In the example, we try to prefetch the next row.

In [134]:
render_code("./prefetch/transpose.cpp", lang="c++", show=["//START", "//END"])

// ./prefetch/transpose.cpp:42-63 (22 lines)
    //START
    //  Iterate each row.
    f_vector *y_iter = T;
    do{
        //  Iterate each column.
        f_vector *ptr_x = y_iter + block;
        f_vector *ptr_y = y_iter + row_size;

        do{

#ifdef ENABLE_PREFETCH
            _mm_prefetch((char*)(ptr_y + row_size),_MM_HINT_T0);
#endif
            swap_block(ptr_x,ptr_y,block);

            ptr_x += block;
            ptr_y += row_size;
        }while (ptr_y < stop_T);

        y_iter += iter_size;
    }while (y_iter < end);
    //END

Now, let's take a look of what's happening!

In [135]:
! cd prefetch; make clean; make
# ! echo "Without prefetch -- the baseline"; ssh htseng@celebi "lscpu | grep Model; cd courses/CS203/demo/memory/prefetch/; ./transpose"
! echo "Without prefetch -- the baseline"
! lscpu | grep Model
! ./prefetch/transpose
! echo "With prefetch"
! ./prefetch/transpose_prefetch

rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
Without prefetch -- the baseline
Model name:                           13th Gen Intel(R) Core(TM) i7-13700
Model:                                183
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.122648 seconds
With prefetch
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.114274 seconds


Let's try a different machine now.

In [136]:
! ssh htseng@xerneas "cd /nfshome/htseng/courses/CSE142/demo/memory/; make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; ssh htseng@xerneas  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose"
! echo "With prefetch";  ssh htseng@xerneas  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose_prefetch"

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
Model name:                           AMD Ryzen 9 5950X 16-Core Processor
Model:                                33
Without prefetch -- the baseline
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.115205 seconds
With prefetch
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.107185 seconds


In [137]:
! ssh htseng@blissey "cd /nfshome/htseng/courses/CSE142/demo/memory/; make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; ssh htseng@blissey  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose"
! echo "With prefetch";  ssh htseng@blissey  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose_prefetch"

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
Model name:                           AMD Ryzen 7 5700X 8-Core Processor
Model:                                33
Without prefetch -- the baseline
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.103619 seconds
With prefetch
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.096393 seconds


In [138]:
! ssh htseng@eevee "cd /nfshome/htseng/courses/CSE142/demo/memory/; make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; ssh htseng@eevee  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose"
! echo "With prefetch";  ssh htseng@eevee  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose_prefetch"

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
make: Entering directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/memory/prefetch'
Model:                              85
Model name:                         Intel(R) Xeon(R) Silver 4108 CPU @ 1.80GHz
Without prefetch -- the baseline
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.199896 seconds
With prefetch
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.172947 seconds



-- It doesn't work always!